In [93]:
512 * 2

1024

### run.py

In [1]:

import argparse
import train
import test
import evaluate

if __name__ == '__main__':
    
    #? IDK what to use these for. Were mentioned in run_experiment.sh
    GPU_IDX=0
    CUDA_DEVICE_ORDER="PCI_BUS_ID"
    CUDA_VISIBLE_DEVICES=GPU_IDX
    
    
    #? setup arg parser to pass in config info
    parser = argparse.ArgumentParser()
    parser.add_argument('--logdir', type=str, default='./log/', help='path to model save dir')
    parser.add_argument('--loglevel', type=str, default='info', help='set level of logger')
    parser.add_argument('--identifier', type=str, default='debug', help='unique run identifier')
    parser.add_argument('--config', type=str, default='./configs/dfaust/config_dfaust.yaml', help='path to yaml config file')
    parser.add_argument('--model_ckpt', type=str, default='000000.pt', help='checkpoint to load')
    parser.add_argument('--fix_random_seed', action='store_true', default=False, help='fix random seed')
    args = parser.parse_args([])

    #? Manually set arg parser values here:
    #? Comment out if you want to use terminal flags above instead (I just got tired of manually entering them in)
    args.logdir = './log/'
    args.loglevel = 'debug'
    args.identifier = 'msr_action_set_transformer_temporal'
    #args.identifier = 'set_transformer_dbug'
    args.config = r'configs\msr-action3d\config_msr_action3d.yaml'
    args.model_ckpt = 'best_000714_0.pt'
    args.fix_random_seed = True
    print('args', args)
    
    print('--------------- starting training')
    #train.main(args) #--identifier $IDENTIFIER --config $CONFIG --logdir $LOGDIR --fix_random_seed
   
    print('--------------- starting testing')

c:\Users\Efran\anaconda3\envs\tpatches_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


args Namespace(config='configs\\msr-action3d\\config_msr_action3d.yaml', fix_random_seed=True, identifier='msr_action_set_transformer_temporal', logdir='./log/', loglevel='debug', model_ckpt='best_000714_0.pt')
--------------- starting training
--------------- starting testing


In [ ]:

import argparse
import train
import test
import evaluate

if __name__ == '__main__':
    
    #? IDK what to use these for. Were mentioned in run_experiment.sh
    GPU_IDX=0
    CUDA_DEVICE_ORDER="PCI_BUS_ID"
    CUDA_VISIBLE_DEVICES=GPU_IDX
    
    
    #? setup arg parser to pass in config info
    parser = argparse.ArgumentParser()
    parser.add_argument('--logdir', type=str, default='./log/', help='path to model save dir')
    parser.add_argument('--loglevel', type=str, default='info', help='set level of logger')
    parser.add_argument('--identifier', type=str, default='debug', help='unique run identifier')
    parser.add_argument('--config', type=str, default='./configs/dfaust/config_dfaust.yaml', help='path to yaml config file')
    parser.add_argument('--model_ckpt', type=str, default='000000.pt', help='checkpoint to load')
    parser.add_argument('--fix_random_seed', action='store_true', default=False, help='fix random seed')
    args = parser.parse_args([])

    #? Manually set arg parser values here:
    #? Comment out if you want to use terminal flags above instead (I just got tired of manually entering them in)
    args.logdir = './log/'
    args.loglevel = 'debug'
    args.identifier = 'msr_action_set_transformer_temporal'
    #args.identifier = 'set_transformer_dbug'
    args.config = r'configs\msr-action3d\config_msr_action3d.yaml'
    args.model_ckpt = 'best_000714_0.pt'
    args.fix_random_seed = True
    print('args', args)
    
    print('--------------- starting training')
    #train.main(args) #--identifier $IDENTIFIER --config $CONFIG --logdir $LOGDIR --fix_random_seed
   
    print('--------------- starting testing')

c:\Users\Efran\anaconda3\envs\tpatches_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


args Namespace(config='configs\\msr-action3d\\config_msr_action3d.yaml', fix_random_seed=True, identifier='msr_action_set_transformer_temporal', logdir='./log/', loglevel='debug', model_ckpt='best_000714_0.pt')
--------------- starting training
--------------- starting testing


### test

In [2]:
import os
import argparse
import i3d_utils
import utils
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from datasets import build_dataloader
import random
import yaml
import sys
import importlib


import logging

def create_basic_logger(logdir, level = 'info'):
    print(f'Using logging level {level} for train.py')
    global logger
    logger = logging.getLogger('test_logger')
    
    #? set logging level
    if level.lower() == 'debug':
        logger.setLevel(logging.DEBUG)
    elif level.lower() == 'info':
        logger.setLevel(logging.INFO)
    elif level.lower() == 'warning':
        logger.setLevel(logging.WARNING)
    elif level.lower() == 'error':
        logger.setLevel(logging.ERROR)
    elif level.lower() == 'critical':
        logger.setLevel(logging.CRITICAL)
    else:
        logger.setLevel(logging.INFO)
    
    #? create handlers
    file_handler = logging.FileHandler(os.path.join(logdir, "log_test.log"))
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    
    stream_handler = logging.StreamHandler()
    #stream_handler.setLevel(logging.INFO)
    #stream_handler.setFormatter(stream_handler)
    logger.addHandler(stream_handler)
    return logger


In [3]:
cfg = yaml.safe_load(open(os.path.join(args.logdir, args.identifier, 'config.yaml')))
logdir = os.path.join(args.logdir, args.identifier)
output_path = os.path.join(logdir, 'results')
os.makedirs(output_path, exist_ok=True)

logger = create_basic_logger(logdir = logdir, level = args.loglevel)

model_path = os.path.join(logdir, args.model_ckpt)

logger.info(f'=================== Starting testing run for {args.identifier}')
logger.info(f'Config: {cfg}')
logger.info(f'Model Path: {model_path}')

if cfg['DATA'].get('name') == 'MSR-Action3D':
    print(True)

=================== Starting testing run for msr_action_set_transformer_temporal
Config: {'DATA': {'data_sampler': 'weighted', 'dataset_path': 'datasets\\data\\MSRAction3D_fps_new\\train', 'dataset_path_test': 'datasets\\data\\MSRAction3D_fps_new\\test', 'frame_skip': 1, 'frames_per_clip': 64, 'gender': 'all', 'n_points': 128, 'name': 'MSR-Action3D', 'noisy_data': {'test': False, 'train': False}, 'shuffle_points': 'fps_each_frame'}, 'MODEL': {'3DMFV': {'n_gaussians': 8}, 'P4TRANSFORMER': {'depth': 10, 'dim': 1024, 'dim_head': 256, 'dropout1': 0.05, 'dropout2': 0.5, 'emb_relu': False, 'heads': 8, 'mlp_dim': 2048, 'nsamples': 32, 'radius': 0.1, 'spatial_stride': 32, 'temporal_kernel_size': 3, 'temporal_stride': 2}, 'PSTNET': {'nsamples': 9, 'radius': 0.1}, 'PST_TRANSFORMER': {'depth': 5, 'dim': 1024, 'dim_head': 256, 'dropout1': 0.0, 'dropout2': 0.5, 'heads': 8, 'mlp_dim': 2048, 'nsamples': 32, 'radius': 0.1, 'spatial_stride': 32, 'temporal_kernel_size': 3, 'temporal_stride': 2}, 'SET_TR

Using logging level debug for train.py
True


In [4]:
if logger == None:
        logger = create_basic_logger(logdir, 'info')

    
batch_size = cfg['TESTING']['batch_size']
frames_per_clip = cfg['DATA']['frames_per_clip']
subset = cfg['TESTING']['set']
pred_output_filename = os.path.join(output_path, subset + '_pred.npy')
json_output_filename = os.path.join(output_path, subset + '_action_segments.json')
data_name = cfg['DATA']['name']

if args.fix_random_seed:
    seed = cfg['seed']
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

test_dataloader, test_dataset = build_dataloader(config=cfg, training=False, shuffle=False, logger=logger)
#test_dataloader, test_dataset = build_dataloader(config=cfg, training=False, shuffle=False)
num_classes = 20

Number of clips in the test set: 102


In [5]:
# setup the model
spec = importlib.util.spec_from_file_location('build_model_from_logdir', os.path.join(logdir, 'models', '__init__.py'))
build_model_from_logdir = importlib.util.module_from_spec(spec)
sys.modules['build_model_from_logdir'] = build_model_from_logdir
spec.loader.exec_module(build_model_from_logdir)
model = build_model_from_logdir.build_model_from_logdir(logdir, cfg['MODEL'], num_classes, frames_per_clip).get()

In [6]:
checkpoints = torch.load(model_path)
model.load_state_dict(checkpoints["model_state_dict"])  # load trained model
model.cuda()
model = nn.DataParallel(model)
model.eval()

n_examples = 0

In [7]:
# Iterate over data.
avg_acc = []
pred_labels_per_video = [[] for _ in range(len(test_dataset))]
logits_per_video = [[] for _ in range(len(test_dataset))]

In [83]:
pred_output_file = []
for test_batchind, data in enumerate(test_dataloader):
    # get the inputs
    if torch.is_tensor(data[0]) == False or torch.is_tensor(data[1]) == False:
        data_list = []
        label_list = []
        for i in data:
            data_list.append(i[1])
            label_list.append(i[0])
            
        inputs = torch.stack(data_list)
        labels = torch.Tensor(label_list)
    else:
        inputs = data[1]
        labels = data[0]
        
    in_channel = cfg['MODEL'].get('in_channel', 3)
    inputs = inputs[:, :, 0:in_channel, :].cuda()
    labels = labels.cuda()
    
    with torch.no_grad():
        out_dict = model(inputs)

    logits = out_dict['pred']
    labels = labels.unsqueeze(1) + torch.zeros((logits.shape[0], logits.shape[2])).cuda()
    labels = labels.to(dtype = torch.long)

    acc = i3d_utils.accuracy_v2(torch.argmax(logits, dim=1), labels)
    avg_acc.append(acc.detach().cpu().numpy())
    
    n_examples += batch_size
    logger.info('batch Acc: {}, [{} / {}]'.format(acc.item(), test_batchind, len(test_dataloader)))
    pred_labels = torch.argmax(logits, 1).detach().cpu().numpy()
    pred_output_file.append({"predicted_labels": pred_labels.tolist(),
                                 "actual_lables": labels.tolist(),
                                 "acc": acc.tolist()})


batch Acc: 0.0, [0 / 51]
batch Acc: 0.0, [1 / 51]
batch Acc: 0.0, [2 / 51]
batch Acc: 0.0, [3 / 51]
batch Acc: 0.0, [4 / 51]
batch Acc: 0.0, [5 / 51]
batch Acc: 0.0, [6 / 51]
batch Acc: 0.0, [7 / 51]
batch Acc: 0.0, [8 / 51]
batch Acc: 0.0, [9 / 51]
batch Acc: 0.0, [10 / 51]
batch Acc: 0.0, [11 / 51]
batch Acc: 0.0, [12 / 51]
batch Acc: 0.0, [13 / 51]
batch Acc: 0.0, [14 / 51]
batch Acc: 0.0, [15 / 51]
batch Acc: 0.0, [16 / 51]
batch Acc: 0.0, [17 / 51]
batch Acc: 0.0, [18 / 51]
batch Acc: 0.0, [19 / 51]
batch Acc: 0.0, [20 / 51]
batch Acc: 0.0, [21 / 51]
batch Acc: 0.0, [22 / 51]
batch Acc: 0.0, [23 / 51]
batch Acc: 50.0, [24 / 51]
batch Acc: 87.5, [25 / 51]
batch Acc: 87.5, [26 / 51]
batch Acc: 0.0, [27 / 51]
batch Acc: 0.0, [28 / 51]
batch Acc: 0.0, [29 / 51]
batch Acc: 0.0, [30 / 51]
batch Acc: 0.0, [31 / 51]
batch Acc: 0.0, [32 / 51]
batch Acc: 0.0, [33 / 51]
batch Acc: 0.0, [34 / 51]
batch Acc: 0.0, [35 / 51]
batch Acc: 0.0, [36 / 51]
batch Acc: 0.0, [37 / 51]
batch Acc: 0.0, [38

In [84]:
pred_output_file

[{'predicted_labels': [[6, 15, 15, 15, 15, 15, 15, 15],
   [15, 15, 15, 15, 15, 15, 6, 15]],
  'actual_lables': [[8, 8, 8, 8, 8, 8, 8, 8],
   [16, 16, 16, 16, 16, 16, 16, 16]],
  'acc': 0.0},
 {'predicted_labels': [[15, 6, 6, 6, 6, 6, 6, 6],
   [6, 6, 6, 6, 15, 15, 15, 15]],
  'actual_lables': [[16, 16, 16, 16, 16, 16, 16, 16],
   [16, 16, 16, 16, 16, 16, 16, 16]],
  'acc': 0.0},
 {'predicted_labels': [[5, 15, 5, 15, 15, 15, 6, 15],
   [6, 6, 6, 6, 6, 6, 6, 6]],
  'actual_lables': [[16, 16, 16, 16, 16, 16, 16, 16],
   [9, 9, 9, 9, 9, 9, 9, 9]],
  'acc': 0.0},
 {'predicted_labels': [[6, 5, 6, 6, 6, 6, 15, 15],
   [15, 15, 15, 15, 15, 15, 15, 15]],
  'actual_lables': [[9, 9, 9, 9, 9, 9, 9, 9], [9, 9, 9, 9, 9, 9, 9, 9]],
  'acc': 0.0},
 {'predicted_labels': [[15, 15, 15, 5, 5, 6, 6, 6],
   [15, 6, 15, 15, 15, 15, 15, 15]],
  'actual_lables': [[9, 9, 9, 9, 9, 9, 9, 9],
   [16, 16, 16, 16, 16, 16, 16, 16]],
  'acc': 0.0},
 {'predicted_labels': [[6, 6, 15, 6, 6, 6, 6, 6],
   [6, 6, 15, 6, 15

In [85]:
import json

In [86]:
os.path.join(args.logdir, args.identifier,'holdout_test_pred_score.json')

'./log/msr_action_set_transformer_temporal\\holdout_test_pred_score.json'

In [87]:
pred_output_file

[{'predicted_labels': [[6, 15, 15, 15, 15, 15, 15, 15],
   [15, 15, 15, 15, 15, 15, 6, 15]],
  'actual_lables': [[8, 8, 8, 8, 8, 8, 8, 8],
   [16, 16, 16, 16, 16, 16, 16, 16]],
  'acc': 0.0},
 {'predicted_labels': [[15, 6, 6, 6, 6, 6, 6, 6],
   [6, 6, 6, 6, 15, 15, 15, 15]],
  'actual_lables': [[16, 16, 16, 16, 16, 16, 16, 16],
   [16, 16, 16, 16, 16, 16, 16, 16]],
  'acc': 0.0},
 {'predicted_labels': [[5, 15, 5, 15, 15, 15, 6, 15],
   [6, 6, 6, 6, 6, 6, 6, 6]],
  'actual_lables': [[16, 16, 16, 16, 16, 16, 16, 16],
   [9, 9, 9, 9, 9, 9, 9, 9]],
  'acc': 0.0},
 {'predicted_labels': [[6, 5, 6, 6, 6, 6, 15, 15],
   [15, 15, 15, 15, 15, 15, 15, 15]],
  'actual_lables': [[9, 9, 9, 9, 9, 9, 9, 9], [9, 9, 9, 9, 9, 9, 9, 9]],
  'acc': 0.0},
 {'predicted_labels': [[15, 15, 15, 5, 5, 6, 6, 6],
   [15, 6, 15, 15, 15, 15, 15, 15]],
  'actual_lables': [[9, 9, 9, 9, 9, 9, 9, 9],
   [16, 16, 16, 16, 16, 16, 16, 16]],
  'acc': 0.0},
 {'predicted_labels': [[6, 6, 15, 6, 6, 6, 6, 6],
   [6, 6, 15, 6, 15

In [81]:
with open(os.path.join(args.logdir, args.identifier,'holdout_test_pred_score.json'), 'w') as f:
    json.dump(pred_output_file, f)

In [89]:
pred_output_file[24]['acc']

50.0

In [52]:
np.save('temp', pred_output_filename)

In [54]:
np.load('temp.npy', allow_pickle=False)

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [55]:
temp_avg_acc = [torch.Tensor(n) for n in avg_acc]
torch.stack(temp_avg_acc, dim=0).mean()

In [59]:
temp_avg_acc = [torch.Tensor(n) for n in avg_acc]
avg_acc_score = torch.stack(temp_avg_acc, dim=0).mean()
logger.info(f'Accuracy score of holdout: {avg_acc_score}%')

Accuracy score of holdout: 4.229323387145996%
